# Day 24 - o1

In [1]:
def parse_initial_line(line):
    """
    Parse lines of the form:
        wireName: value
    Example:
        x00: 1
    Returns wireName, value (as integer)
    """
    # Split on ':'
    left, right = line.split(':')
    wire_name = left.strip()
    val_str = right.strip()
    return wire_name, int(val_str)

def parse_gate_line(line):
    """
    Parse lines of the form:
        inWire1 OP inWire2 -> outWire
    Where OP is one of: AND, OR, XOR
    Returns: (in1, operation, in2, out)
    Example:
        x00 AND y00 -> z00
    """
    # Something like: "x00 AND y00 -> z00"
    left, out_wire = line.split('->')
    out_wire = out_wire.strip()  # e.g. 'z00'

    # left might look like "x00 AND y00"
    parts = left.strip().split()
    # parts = [inWire1, OP, inWire2]
    in_wire1, operation, in_wire2 = parts
    return in_wire1, operation, in_wire2, out_wire

def gate_operation(op, val1, val2):
    """
    Given an operation (AND, OR, XOR) and two known boolean/int values 0 or 1,
    return the result (0 or 1). Assumes val1 and val2 are already 0 or 1.
    """
    if op == 'AND':
        return 1 if (val1 == 1 and val2 == 1) else 0
    elif op == 'OR':
        return 1 if (val1 == 1 or val2 == 1) else 0
    elif op == 'XOR':
        return 1 if (val1 != val2) else 0
    else:
        raise ValueError(f"Unknown gate operation: {op}")

def main():
    # Read all lines from input.txt
    with open('input.txt', 'r') as f:
        lines = [l.strip() for l in f if l.strip()]

    # We will separate the input into two parts:
    # 1) The initial wire values
    # 2) The gate definitions
    #
    # The puzzle typically has something like:
    #
    #   x00: 1
    #   x01: 1
    #   x02: 1
    #   y00: 0
    #   y01: 1
    #   y02: 0
    #
    #   x00 AND y00 -> z00
    #   x01 XOR y01 -> z01
    #   x02 OR y02 -> z02
    #
    # There's often a blank line separating them, but the puzzle might not rely strictly
    # on a single blank line. Instead, we can detect which lines look like "wire: value"
    # and which lines look like "wire1 OP wire2 -> wire3".

    initial_values = {}
    gates = []

    # We’ll detect format by checking if a line contains '->'. 
    # If it does, we treat it as a gate definition. Otherwise, an initial assignment.
    for line in lines:
        if '->' in line:
            # Gate definition
            in1, op, in2, out = parse_gate_line(line)
            gates.append((in1, op, in2, out))
        else:
            # Initial value definition
            wire_name, val = parse_initial_line(line)
            initial_values[wire_name] = val

    # We store known wire values in a dictionary
    wire_values = dict(initial_values)

    # We will repeatedly try to evaluate gates until no more changes occur
    changed = True
    while changed:
        changed = False
        for (in1, op, in2, out_wire) in gates:
            # Check if we already know the output. If so, skip unless it might change.
            # Because once stable, it won't change. But let's re-check to see if
            # it's possible to compute a new value.
            if out_wire in wire_values:
                # We already have a value. Since the puzzle states no loops,
                # that value should not change. But let's skip anyway.
                continue

            # We need both inputs known to compute an output
            if in1 in wire_values and in2 in wire_values:
                val_in1 = wire_values[in1]
                val_in2 = wire_values[in2]
                # Evaluate
                out_val = gate_operation(op, val_in1, val_in2)
                wire_values[out_wire] = out_val
                changed = True

    # Now gather all wires whose names start with 'z'
    # The wire name pattern is something like z00, z01, z02, z10, z11, ...
    # We want to sort them by their numeric suffix so that z00 is the least significant bit,
    # z01 is next, and so on.
    z_wires = []
    for w in wire_values:
        if w.startswith('z'):
            # The rest of the name is the numeric part, e.g. "00", "01", ...
            # Convert that to an integer so we can sort properly.
            suffix = w[1:]  # e.g. skip the 'z'
            # If suffix might be something like '05', int(...) will handle that.
            try:
                idx = int(suffix)
                z_wires.append((idx, w))
            except ValueError:
                # If there's some unexpected naming, ignore it or handle accordingly
                pass

    # Sort by the numeric suffix
    z_wires.sort(key=lambda x: x[0])

    # Build the binary string from least significant bit (z00) to largest (zXX)
    # The puzzle states that z00 is the least significant bit, then z01, etc.
    bits = []
    for idx, wire_name in z_wires:
        bit_val = wire_values.get(wire_name, 0)  # default to 0 if not known
        bits.append(str(bit_val))

    # bits[0] is least significant => the binary number is bits[-1] as MSB
    # The puzzle’s example collects them so that z02 is the most significant bit in the example.
    # However, if we read the puzzle carefully, the final example’s explanation:
    #   "the wires in this system have the following values ... Combining the bits from all wires
    #    starting with z produces the binary number 0011111101000"
    #
    # That means z00 was the rightmost bit of that number (least significant).
    #
    # So we need to reverse them at the end when constructing the standard binary notation
    # or be mindful that bits[0] is LSB and bits[n-1] is MSB.
    #
    # Usually, when we do int(binary_str, 2), the leftmost digit is the most significant bit.
    # So let's build the binary string so that bits[-1] is on the left. That means we reverse.
    #
    # Example: if bits = ['0','1','1'] => z00=0, z01=1, z02=1
    # The binary integer with z02 as MSB => 110 in normal binary notation. 
    #
    # So we do:
    binary_str = ''.join(reversed(bits)) if bits else '0'

    # Convert to decimal
    result_decimal = int(binary_str, 2) if binary_str else 0

    print(result_decimal)

if __name__ == "__main__":
    main()


57344080719736


## Part 2

In [2]:
import itertools

def parse_initial_line(line):
    # Same as Part One
    wire, val = line.split(':')
    return wire.strip(), int(val.strip())

def parse_gate_line(line):
    # Same as Part One
    left, out_wire = line.split('->')
    out_wire = out_wire.strip()
    parts = left.strip().split()
    in1, op, in2 = parts
    return in1, op, in2, out_wire

def gate_operation(op, val1, val2):
    # Same as Part One
    if op == 'AND':
        return 1 if (val1 and val2) else 0
    elif op == 'OR':
        return 1 if (val1 or val2) else 0
    elif op == 'XOR':
        return 1 if (val1 != val2) else 0
    else:
        raise ValueError(f"Unknown operation {op}")

def simulate_circuit(initial_values, gates):
    """
    Given:
      initial_values: dict of {wire_name: 0|1}
      gates: list of (in1, op, in2, outWire)
    Returns:
      A dict {wire_name: 0|1} of all wires that become determined.
    """
    wire_values = dict(initial_values)

    changed = True
    while changed:
        changed = False
        for (in1, op, in2, out_wire) in gates:
            if out_wire in wire_values:
                # Already has a value; with no feedback loops, it won't change
                continue
            if in1 in wire_values and in2 in wire_values:
                val_out = gate_operation(op, wire_values[in1], wire_values[in2])
                wire_values[out_wire] = val_out
                changed = True

    return wire_values

def is_adder_correct(num_bits, wire_values):
    """
    Check if the 'z' wires represent the sum of the 'x' wires and 'y' wires.
    For example, if we have num_bits bits each for x and y, then:
      - x00 is LSB of x, x01 next, etc.
      - y00 is LSB of y, ...
      - z?? is the sum.

    This function returns True if the z-wires in `wire_values`
    indeed match x + y for *this particular* set of initial values.

    Note: In a full check, you'd want to do this for many input patterns
    or all 2^(2*num_bits) possible patterns (if feasible).
    """
    # Extract the x bits
    x_val = 0
    for b in range(num_bits):
        wire_name = f"x{b:02d}"
        bit_val = wire_values.get(wire_name, 0)
        x_val |= (bit_val << b)

    # Extract the y bits
    y_val = 0
    for b in range(num_bits):
        wire_name = f"y{b:02d}"
        bit_val = wire_values.get(wire_name, 0)
        y_val |= (bit_val << b)

    # Sum
    sum_val = x_val + y_val

    # Extract bits from z
    # We might expect up to num_bits+1 bits, or more, depending on the puzzle.
    # In an adder, if there are N bits of x, N bits of y, you typically have up to N+1 bits of z.
    # But your puzzle might have more wires than that. We'll check at least num_bits+1.
    # (Or parse from the puzzle how many z bits to expect.)
    max_z = num_bits + 2  # or more, depending on puzzle
    computed_sum = 0
    for b in range(max_z):
        wire_name = f"z{b:02d}"
        bit_val = wire_values.get(wire_name, 0)
        computed_sum |= (bit_val << b)

    return (computed_sum == sum_val)

def swap_gates(gate1, gate2):
    """
    gate1, gate2 are tuples like (in1, op, in2, outWire).
    We swap their 'outWire' fields and return new copies of both gates.
    """
    (in1a, opa, in2a, outa) = gate1
    (in1b, opb, in2b, outb) = gate2
    # Swap outa <-> outb
    gate1_swapped = (in1a, opa, in2a, outb)
    gate2_swapped = (in1b, opb, in2b, outa)
    return gate1_swapped, gate2_swapped

def main():
    # 1) Read lines
    with open('input.txt') as f:
        lines = [l.strip() for l in f if l.strip()]

    # 2) Separate into initial values vs gates
    initial_values = {}
    gates = []
    for line in lines:
        if '->' in line:
            g = parse_gate_line(line)
            gates.append(g)
        else:
            w, v = parse_initial_line(line)
            initial_values[w] = v

    # We assume we know how many bits are in x, y, etc. 
    # (For your puzzle, parse or deduce the maximum index among x??, y??, z??.)
    num_bits = 12  # Example guess, might need to detect automatically

    # 3) Keep a copy of the original gates for reference
    original_gates = list(gates)

    # 4) Identify all gates' outputs for potential swapping
    #    We'll store them as (index, gate) so we can refer to them by index
    all_gates_indexed = list(enumerate(original_gates))

    # 5) We want exactly 4 disjoint pairs of gates to swap. Let's do a naive approach:
    #    Step 1: choose 8 distinct gates out of all.
    #    Step 2: figure out how to pair them up in 4 pairs (any perfect matching).
    #    Step 3: swap those outputs, test correctness.
    #
    #    If your system is large, this can be very expensive.
    #    A typical puzzle might be crafted so that it's still solvable with some optimizations.

    import math

    # We'll write a helper that, given 8 distinct gate indices, tries all ways to split them into 4 disjoint pairs.
    def all_pairings(eight_indices):
        """
        Given a list of 8 distinct indices, yield all ways to partition them into 4 disjoint pairs.
        For example, if eight_indices = [1,2,3,4,5,6,7,8],
        one pairing might be (1,2), (3,4), (5,6), (7,8).
        Another might be (1,3), (2,4), (5,7), (6,8), etc.
        """
        if not eight_indices:
            yield []
            return
        first = eight_indices[0]
        for i in range(1, len(eight_indices)):
            second = eight_indices[i]
            pair = (first, second)
            remaining = eight_indices[1:i] + eight_indices[i+1:]
            for rest in all_pairings(remaining):
                yield [pair] + rest

    # We'll write a small function to "apply swaps" (i.e., produce swapped gates) given a set of pairs.
    def apply_swaps(gates_in, pairs):
        gates_out = list(gates_in)
        for (i, j) in pairs:
            gates_out[i], gates_out[j] = swap_gates(gates_out[i], gates_out[j])
        return gates_out

    # 6) Actually do the search
    # Because we know exactly 4 pairs => we need 8 distinct gates.
    # We'll do a combination of all gates of size 8, and for each, try all pairings.
    # This is definitely brute force, so be cautious on large inputs!

    gates_count = len(original_gates)
    from itertools import combinations

    # For demonstration, let's try a small or partial search.
    # If your puzzle is big, you might need a more advanced approach or partial checks.
    correct_swaps = None
    correct_swapped_gates = None

    # We'll do a small set of test vectors for x,y to check correctness quickly.
    # For an N-bit adder, let's test a handful of inputs (e.g., 0, 1, max, random).
    # The more tests you do, the more sure you are that you found the real fix.

    test_vectors = []
    # Some corner cases
    test_vectors.append(([0]*num_bits, [0]*num_bits))             # x=0, y=0
    test_vectors.append(([1]*num_bits, [0]*num_bits))             # x=all1, y=0
    test_vectors.append(([0]*num_bits, [1]*num_bits))             # x=0, y=all1
    # A few random patterns
    import random
    for _ in range(5):
        x_rand = [random.randint(0,1) for _ in range(num_bits)]
        y_rand = [random.randint(0,1) for _ in range(num_bits)]
        test_vectors.append((x_rand, y_rand))

    def load_test_vector(x_bits, y_bits):
        """
        Overwrite wire values 'x00'..'xNN', 'y00'..'yNN' in initial_values and return a copy,
        because we want to simulate the circuit with these bits as input.
        """
        test_vals = dict(initial_values)
        for b in range(num_bits):
            test_vals[f"x{b:02d}"] = x_bits[b]
            test_vals[f"y{b:02d}"] = y_bits[b]
        return test_vals

    def check_adder_for_all_tests(gates_candidate):
        """
        Return True if for every (x_bits, y_bits) in test_vectors,
        the circuit with gates_candidate produces a correct sum.
        """
        for (x_bits, y_bits) in test_vectors:
            # 1) Copy initial values, overriding x?? and y?? from test
            test_vals = load_test_vector(x_bits, y_bits)
            # 2) Simulate
            final_vals = simulate_circuit(test_vals, gates_candidate)
            # 3) Check correctness
            if not is_adder_correct(num_bits, final_vals):
                return False
        return True

    # Now brute force:
    # 1. Choose 8 distinct gate indices
    for eight_indices in combinations(range(gates_count), 8):
        # 2. For each possible pairing of those 8
        for pairing in all_pairings(list(eight_indices)):
            # pairing is a list of 4 pairs: [ (i1,j1), (i2,j2), (i3,j3), (i4,j4) ]
            swapped_gates = apply_swaps(original_gates, pairing)
            if check_adder_for_all_tests(swapped_gates):
                # We found a solution!
                correct_swaps = pairing
                correct_swapped_gates = swapped_gates
                break
        if correct_swaps is not None:
            break

    if correct_swaps is None:
        print("No swap combination found that fixes the adder (in this brute force search)!")
        return

    # If we found a solution, we want to list the output wires that got swapped.
    # correct_swaps is something like [ (gateIdx1, gateIdx2), (gateIdx3, gateIdx4), ... ]
    # Let's collect the wire names from each pair.
    swapped_wire_names = []
    for (i, j) in correct_swaps:
        # original gates:
        in1a, opa, in2a, outa = original_gates[i]
        in1b, opb, in2b, outb = original_gates[j]
        # The puzzle wants the wire names that were swapped
        # i.e. outa and outb
        swapped_wire_names.append(outa)
        swapped_wire_names.append(outb)

    # Now sort them and output
    swapped_wire_names.sort()
    answer = ",".join(swapped_wire_names)
    print("Swapped wire names:", answer)

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [ ]:
import itertools
import random

def parse_initial_line(line):
    """
    Parse 'wireName: value' lines, e.g. 'x00: 1'.
    Returns (wireName, intValue).
    """
    left, right = line.split(':')
    return left.strip(), int(right.strip())

def parse_gate_line(line):
    """
    Parse lines like: 'x00 AND y00 -> z05'
    Returns (inWire1, operation, inWire2, outWire).
    """
    left, out_wire = line.split('->')
    out_wire = out_wire.strip()
    parts = left.strip().split()  # e.g. ['x00', 'AND', 'y00']
    return parts[0], parts[1], parts[2], out_wire

def gate_operation(op, val1, val2):
    """
    Given op in {AND, OR, XOR} and two bits (0|1),
    returns the resulting bit (0|1).
    """
    if op == "AND":
        return 1 if (val1 == 1 and val2 == 1) else 0
    elif op == "OR":
        return 1 if (val1 == 1 or val2 == 1) else 0
    elif op == "XOR":
        return 1 if (val1 != val2) else 0
    else:
        raise ValueError(f"Unknown operation: {op}")

def simulate_circuit(initial_values, gates):
    """
    Run the circuit until no new wire values appear.
    gates: list of (in1, op, in2, outWire).
    initial_values: dict {wireName: 0|1}.
    Return a dict of all final wire values.
    """
    wire_values = dict(initial_values)  # copy

    changed = True
    while changed:
        changed = False
        for (in1, op, in2, out_wire) in gates:
            if out_wire in wire_values:
                # Already set; with no loops, it won't change again.
                continue
            if in1 in wire_values and in2 in wire_values:
                val_out = gate_operation(op, wire_values[in1], wire_values[in2])
                wire_values[out_wire] = val_out
                changed = True

    return wire_values

def is_adder_correct(num_bits, wire_values):
    """
    Check if z?? == (x?? + y??) for the given wire_values.
    We interpret x00..x(num_bits-1) as one binary number,
    y?? as second, z?? as sum.
    Return True if matches x + y, else False.
    """
    # Reconstruct integer x
    x_val = 0
    for b in range(num_bits):
        bit = wire_values.get(f"x{b:02d}", 0)
        x_val |= (bit << b)

    # Reconstruct integer y
    y_val = 0
    for b in range(num_bits):
        bit = wire_values.get(f"y{b:02d}", 0)
        y_val |= (bit << b)

    # Reconstruct integer z (we allow up to num_bits+1 or more).
    # Adjust as needed for your puzzle—if you have more z bits, you can check them all.
    sum_val = x_val + y_val
    z_val = 0
    max_z_bits = num_bits + 2  # or parse from puzzle if you know exactly how many
    for b in range(max_z_bits):
        bit = wire_values.get(f"z{b:02d}", 0)
        z_val |= (bit << b)

    return (z_val == sum_val)

def swap_gates(gateA, gateB):
    """
    Swap the 'outWire' fields of two gate definitions.
    gateA, gateB: (in1, op, in2, outWire)
    Returns new versions of gateA and gateB with swapped outputs.
    """
    in1a, opa, in2a, outA = gateA
    in1b, opb, in2b, outB = gateB
    # Swap outA <-> outB
    newA = (in1a, opa, in2a, outB)
    newB = (in1b, opb, in2b, outA)
    return newA, newB

def main():
    # --- 1) Read and parse input ---
    with open('input.txt', 'r') as f:
        lines = [l.strip() for l in f if l.strip()]

    initial_values = {}
    gates = []
    for line in lines:
        if '->' in line:
            g = parse_gate_line(line)
            gates.append(g)
        else:
            w, v = parse_initial_line(line)
            initial_values[w] = v

    # Estimate how many bits we might have for x or y (puzzle-specific).
    # If you see x00..x31, y00..y31, then num_bits could be 32, etc.
    # For demonstration, we'll just guess or parse it. Example:
    num_bits = 12  # adjust to match your puzzle's largest x??, y??

    # Keep an original copy of the gates
    original_gates = list(gates)

    # We expect exactly 4 swapped pairs => 8 swapped wires total.
    # We'll do a *naive* search for small puzzles. For very large puzzles, you'd optimize or use a solver.

    # A set of test vectors to verify correctness quickly (instead of all 2^(2*num_bits) possibilities!)
    test_vectors = []
    # Some corner cases:
    test_vectors.append(([0]*num_bits, [0]*num_bits))       # x=0, y=0
    test_vectors.append(([1]*num_bits, [0]*num_bits))       # x=all1, y=0
    test_vectors.append(([0]*num_bits, [1]*num_bits))       # x=0, y=all1
    # A few random patterns:
    for _ in range(5):
        x_rand = [random.randint(0,1) for _ in range(num_bits)]
        y_rand = [random.randint(0,1) for _ in range(num_bits)]
        test_vectors.append((x_rand, y_rand))

    def load_test_vector_into(initial_vals, x_bits, y_bits):
        """
        Given base initial_vals, set x?? and y?? accordingly. Return a copy.
        """
        vals = dict(initial_vals)
        for b in range(num_bits):
            vals[f"x{b:02d}"] = x_bits[b]
            vals[f"y{b:02d}"] = y_bits[b]
        return vals

    def all_tests_pass(gates_candidate):
        """
        Returns True if for all test_vectors, the circuit sums correctly.
        """
        for (x_bits, y_bits) in test_vectors:
            test_init = load_test_vector_into(initial_values, x_bits, y_bits)
            out_vals = simulate_circuit(test_init, gates_candidate)
            if not is_adder_correct(num_bits, out_vals):
                return False
        return True

    # Identify each gate by index so we can pick them in pairs
    gate_indices = list(range(len(original_gates)))

    # We'll create a helper to generate all ways to form 4 disjoint pairs from 8 chosen gates.
    def pairings_of_8_indices(lst):
        """
        Given 8 distinct indices, yield all ways to partition them into 4 pairs.
        """
        if not lst:
            yield []
            return
        first = lst[0]
        for i in range(1, len(lst)):
            second = lst[i]
            pair = (first, second)
            remainder = lst[1:i] + lst[i+1:]
            for rest in pairings_of_8_indices(remainder):
                yield [pair] + rest

    found_solution = False
    solution_pairs = None

    # We'll brute force: pick exactly 8 gates, then pair them up in all possible ways.
    # For each pairing, swap outputs, test, etc.
    # This is obviously big-O heavy for large circuits—but it’s illustrative.
    from itertools import combinations

    for combo8 in combinations(gate_indices, 8):
        combo8 = list(combo8)
        for four_pairs in pairings_of_8_indices(combo8):
            # Make a copy of original gates
            swapped_version = list(original_gates)
            # Apply each of the 4 swaps
            for (i, j) in four_pairs:
                swapped_version[i], swapped_version[j] = swap_gates(swapped_version[i], swapped_version[j])

            # Test
            if all_tests_pass(swapped_version):
                found_solution = True
                solution_pairs = four_pairs
                break
        if found_solution:
            break

    if not found_solution:
        print("No 4-pair swap combination found (in this brute force) that fixes the adder!")
        return

    # If we found a solution, gather the wire names from those 8 gates' outputs.
    swapped_wire_names = []
    for (i, j) in solution_pairs:
        # original gates i, j
        in1a, opa, in2a, outA = original_gates[i]
        in1b, opb, in2b, outB = original_gates[j]
        swapped_wire_names.extend([outA, outB])

    swapped_wire_names.sort()  # sort alphabetically
    answer = ",".join(swapped_wire_names)
    print(answer)

if __name__ == "__main__":
    main()


In [4]:
import itertools, random

def parse_init(line):
    w,v = line.split(':'); return w.strip(),int(v.strip())

def parse_gate(line):
    left,right=line.split('->');out=right.strip()
    p=left.strip().split();return p[0],p[1],p[2],out

def gate_op(op,a,b):
    if op=="AND": return a&b
    if op=="OR":  return a|b
    if op=="XOR": return a^b
    raise ValueError("Bad op")

def simulate(init_vals,gates):
    vals=dict(init_vals)
    changed=True
    while changed:
        changed=False
        for i1,op,i2,out in gates:
            if out in vals:continue
            if i1 in vals and i2 in vals:
                vals[out]=gate_op(op,vals[i1],vals[i2])
                changed=True
    return vals

def adder_ok(n,ws):
    # Reconstruct x,y,z
    x=y=z=0
    for i in range(n): x|=(ws.get(f"x{i:02d}",0)<<i)
    for i in range(n): y|=(ws.get(f"y{i:02d}",0)<<i)
    for i in range(n+2): z|=(ws.get(f"z{i:02d}",0)<<i)
    return (x+y)==z

def swap_outs(g1,g2):
    i1,o1,i2,o2=g1[0],g1[1],g1[2],g1[3]
    j1,o3,j2,o4=g2[0],g2[1],g2[2],g2[3]
    return (i1,o1,i2,o4),(j1,o3,j2,o2)

def main():
    with open("input.txt") as f:
        lines=[l.strip() for l in f if l.strip()]
    inits,gates={},[]
    for ln in lines:
        if "->" in ln:gates.append(parse_gate(ln))
        else:
            w,v=parse_init(ln)
            inits[w]=v

    # Adjust as needed: how many bits for x,y
    num_bits=12

    orig_gates=list(gates)
    idxes=range(len(gates))

    # Some quick tests (instead of all possible inputs!)
    def all_tests_ok(g):
        # corner tests
        tv=[ ([0]*num_bits,[0]*num_bits),
             ([1]*num_bits,[0]*num_bits),
             ([0]*num_bits,[1]*num_bits),
             ([1]*num_bits,[1]*num_bits) ]
        for xbits,ybits in tv:
            vals=dict(inits)
            for i in range(num_bits): vals[f"x{i:02d}"]=xbits[i]
            for i in range(num_bits): vals[f"y{i:02d}"]=ybits[i]
            out=simulate(vals,g)
            if not adder_ok(num_bits,out):return False
        return True

    found=False
    ans=[]
    # We want exactly 8 distinct gates -> 4 pairs
    def pair8(aa):
        if not aa: yield []
        else:
            f=aa[0]
            for i in range(1,len(aa)):
                s=aa[i]
                for rest in pair8(aa[1:i]+aa[i+1:]):
                    yield [(f,s)]+rest

    for combo in itertools.combinations(idxes,8):
        combo=list(combo)
        for pairs in pair8(combo):
            newg=list(orig_gates)
            for (i,j) in pairs:
                newg[i],newg[j]=swap_outs(newg[i],newg[j])
            if all_tests_ok(newg):
                found=True
                for (i,j) in pairs:
                    ans+=[orig_gates[i][3],orig_gates[j][3]]
                break
        if found: break

    ans.sort()
    print(",".join(ans))

if __name__=="__main__":
    main()


bbq,jjs,nkr,ppw,qnv,tkn,wcw,z34


In [ ]:
import itertools, random

def parse_init(line):
    w,v = line.split(':')
    return w.strip(), int(v.strip())

def parse_gate(line):
    left, right = line.split('->')
    out_wire = right.strip()
    parts = left.strip().split()
    return parts[0], parts[1], parts[2], out_wire  # (in1, op, in2, out)

def gate_op(op, a, b):
    if op == "AND": return a & b
    if op == "OR":  return a | b
    if op == "XOR": return a ^ b
    raise ValueError("Unknown op")

def simulate(init_vals, gates):
    vals = dict(init_vals)
    changed = True
    while changed:
        changed = False
        for (i1, op, i2, outw) in gates:
            if outw in vals: 
                continue
            if i1 in vals and i2 in vals:
                vals[outw] = gate_op(op, vals[i1], vals[i2])
                changed = True
    return vals

def adder_ok(num_bits, wire_vals):
    """
    Checks if z?? == x?? + y?? (for bits 0..num_bits-1).
    We'll consider up to (num_bits+2) bits in z.
    """
    x = y = z = 0
    for i in range(num_bits):
        x |= (wire_vals.get(f"x{i:02d}", 0) << i)
        y |= (wire_vals.get(f"y{i:02d}", 0) << i)
    # Sum:
    s = x + y
    # Reconstruct z from up to num_bits+2 bits
    for i in range(num_bits+2):
        z |= (wire_vals.get(f"z{i:02d}", 0) << i)
    return (z == s)

def swap_outputs(g1, g2):
    """
    Swap the output wires of gate g1 and g2.
    Each gate is (in1, op, in2, outWire).
    """
    (i1a, opa, i2a, outA), (i1b, opb, i2b, outB) = g1, g2
    return (i1a, opa, i2a, outB), (i1b, opb, i2b, outA)

def main():
    # --- 1) Read input ---
    with open("input.txt") as f:
        lines = [l.strip() for l in f if l.strip()]

    inits, gates = {}, []
    for ln in lines:
        if '->' in ln:
            gates.append(parse_gate(ln))
        else:
            w, v = parse_init(ln)
            inits[w] = v

    # Adjust this to match your puzzle's bit-width
    num_bits = 12  

    # Keep original gates:
    orig_gates = list(gates)
    gate_indices = list(range(len(orig_gates)))

    # We'll do a quick test set (instead of testing all possible inputs)
    test_vectors = [
        ([0]*num_bits, [0]*num_bits),
        ([1]*num_bits, [0]*num_bits),
        ([0]*num_bits, [1]*num_bits)
    ]
    # add some random combos
    for _ in range(5):
        xb = [random.randint(0,1) for _ in range(num_bits)]
        yb = [random.randint(0,1) for _ in range(num_bits)]
        test_vectors.append((xb,yb))

    def check_gates(gate_list):
        # For each (x_bits, y_bits), load them into inits, then simulate, check adder
        for (xb, yb) in test_vectors:
            tv = dict(inits)
            for i in range(num_bits): 
                tv[f"x{i:02d}"] = xb[i]
                tv[f"y{i:02d}"] = yb[i]
            out_vals = simulate(tv, gate_list)
            if not adder_ok(num_bits, out_vals):
                return False
        return True

    # Helper to form all ways to pair 8 distinct indices into 4 pairs
    def pairings_8(lst):
        # If empty => yield empty
        if not lst:
            yield []
            return
        first = lst[0]
        for i in range(1, len(lst)):
            second = lst[i]
            pair = (first, second)
            rem = lst[1:i] + lst[i+1:]
            for rest in pairings_8(rem):
                yield [pair] + rest

    found_solution = False
    solution_pairs = None

    # We know exactly 8 gates are swapped (4 pairs). We'll brute force:
    from itertools import combinations

    for c8 in combinations(gate_indices, 8):  # pick 8 distinct gates
        c8 = list(c8)
        for pset in pairings_8(c8):
            # make swapped copy
            swapped = list(orig_gates)
            for (i, j) in pset:
                swapped[i], swapped[j] = swap_outputs(swapped[i], swapped[j])
            if check_gates(swapped):
                found_solution = True
                solution_pairs = pset
                break
        if found_solution:
            break

    if not found_solution:
        print("No solution found (4 swapped pairs) with this brute force!")
        return

    # Gather the 8 swapped wire names
    swapped_wires = []
    for (i, j) in solution_pairs:
        # original gates i,j
        outA = orig_gates[i][3]
        outB = orig_gates[j][3]
        swapped_wires.append(outA)
        swapped_wires.append(outB)

    swapped_wires.sort()
    print(",".join(swapped_wires))

if __name__ == "__main__":
    main()


In [ ]:
import itertools, random

def parse_init(line):
    w,v = line.split(':')
    return w.strip(), int(v.strip())

def parse_gate(line):
    left, right = line.split('->')
    out_wire = right.strip()
    parts = left.strip().split()
    return parts[0], parts[1], parts[2], out_wire  # (in1, op, in2, out)

def gate_op(op, a, b):
    if op == "AND": return a & b
    if op == "OR":  return a | b
    if op == "XOR": return a ^ b
    raise ValueError("Unknown op")

def simulate(init_vals, gates):
    vals = dict(init_vals)
    changed = True
    while changed:
        changed = False
        for (i1, op, i2, outw) in gates:
            if outw in vals: 
                continue
            if i1 in vals and i2 in vals:
                vals[outw] = gate_op(op, vals[i1], vals[i2])
                changed = True
    return vals

def adder_ok(num_bits, wire_vals):
    """
    Checks if z?? == x?? + y?? (for bits 0..num_bits-1).
    We'll consider up to (num_bits+2) bits in z.
    """
    x = y = z = 0
    for i in range(num_bits):
        x |= (wire_vals.get(f"x{i:02d}", 0) << i)
        y |= (wire_vals.get(f"y{i:02d}", 0) << i)
    # Sum:
    s = x + y
    # Reconstruct z from up to num_bits+2 bits
    for i in range(num_bits+2):
        z |= (wire_vals.get(f"z{i:02d}", 0) << i)
    return (z == s)

def swap_outputs(g1, g2):
    """
    Swap the output wires of gate g1 and g2.
    Each gate is (in1, op, in2, outWire).
    """
    (i1a, opa, i2a, outA), (i1b, opb, i2b, outB) = g1, g2
    return (i1a, opa, i2a, outB), (i1b, opb, i2b, outA)

def main():
    # --- 1) Read input ---
    with open("input.txt") as f:
        lines = [l.strip() for l in f if l.strip()]

    inits, gates = {}, []
    for ln in lines:
        if '->' in ln:
            gates.append(parse_gate(ln))
        else:
            w, v = parse_init(ln)
            inits[w] = v

    # Adjust this to match your puzzle's bit-width
    num_bits = 12  

    # Keep original gates:
    orig_gates = list(gates)
    gate_indices = list(range(len(orig_gates)))

    # We'll do a quick test set (instead of testing all possible inputs)
    test_vectors = [
        ([0]*num_bits, [0]*num_bits),
        ([1]*num_bits, [0]*num_bits),
        ([0]*num_bits, [1]*num_bits)
    ]
    # add some random combos
    for _ in range(5):
        xb = [random.randint(0,1) for _ in range(num_bits)]
        yb = [random.randint(0,1) for _ in range(num_bits)]
        test_vectors.append((xb,yb))

    def check_gates(gate_list):
        # For each (x_bits, y_bits), load them into inits, then simulate, check adder
        for (xb, yb) in test_vectors:
            tv = dict(inits)
            for i in range(num_bits): 
                tv[f"x{i:02d}"] = xb[i]
                tv[f"y{i:02d}"] = yb[i]
            out_vals = simulate(tv, gate_list)
            if not adder_ok(num_bits, out_vals):
                return False
        return True

    # Helper to form all ways to pair 8 distinct indices into 4 pairs
    def pairings_8(lst):
        # If empty => yield empty
        if not lst:
            yield []
            return
        first = lst[0]
        for i in range(1, len(lst)):
            second = lst[i]
            pair = (first, second)
            rem = lst[1:i] + lst[i+1:]
            for rest in pairings_8(rem):
                yield [pair] + rest

    found_solution = False
    solution_pairs = None

    # We know exactly 8 gates are swapped (4 pairs). We'll brute force:
    from itertools import combinations

    for c8 in combinations(gate_indices, 8):  # pick 8 distinct gates
        c8 = list(c8)
        for pset in pairings_8(c8):
            # make swapped copy
            swapped = list(orig_gates)
            for (i, j) in pset:
                swapped[i], swapped[j] = swap_outputs(swapped[i], swapped[j])
            if check_gates(swapped):
                found_solution = True
                solution_pairs = pset
                break
        if found_solution:
            break

    if not found_solution:
        print("No solution found (4 swapped pairs) with this brute force!")
        return

    # Gather the 8 swapped wire names
    swapped_wires = []
    for (i, j) in solution_pairs:
        # original gates i,j
        outA = orig_gates[i][3]
        outB = orig_gates[j][3]
        swapped_wires.append(outA)
        swapped_wires.append(outB)

    swapped_wires.sort()
    print(",".join(swapped_wires))

if __name__ == "__main__":
    main()
